# 데이터분석을 통한 서울시 구별 전기차 충전소 배치 제안
- 데이터 정리

## 라이브러리 로드

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import requests
import json
import plotly.express as px

# 지수표현식 제거하기
pd.options.display.float_format = '{:.5f}'.format

# 그래프에 한글 시각화
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

## 데이터로드

### 서울시 전기차 데이터
- 서울시의 구별 전기차 수 데이터가 필요하다.

In [45]:
el_car = pd.read_csv("data/서울시친환경자동차220228.csv", encoding='cp949')
print(el_car.shape)
el_car.head(2)

(189586, 7)


,기준년월,사용본거지시읍면동_행정동기준,차명,연료,최초등록일,현소유자의출생년도,제원관리번호
0,202202,서울특별시 강남구 개포1동,토요타 RAV4 Hybrid AWD,하이브리드(휘발유+전기),20200910,1977.00000,01020007000041319
1,202202,서울특별시 강남구 개포1동,그랜저 하이브리드,하이브리드(휘발유+전기),20170710,1959.00000,A0810010800451317


In [46]:
# 연료가 전기인 데이터만 추출
el_car = el_car.loc[el_car['연료'] == '전기', ['기준년월', '사용본거지시읍면동_행정동기준', '차명', '연료']]
el_car.head()

,기준년월,사용본거지시읍면동_행정동기준,차명,연료
6,202202,서울특별시 강남구 개포1동,Model 3 Standard Range Plus,전기
48,202202,서울특별시 강남구 개포2동,Model 3 Standard Range Plus,전기
58,202202,서울특별시 강남구 개포2동,CHEVROLET BOLT EV,전기
61,202202,서울특별시 강남구 개포2동,GV60,전기
64,202202,서울특별시 강남구 개포2동,Model Y Long Range,전기


In [47]:
# 컬럼명 변경
el_car = el_car.rename(columns={"사용본거지시읍면동_행정동기준":"주소"})
el_car.sample(2)

,기준년월,주소,차명,연료
41686,202202,서울특별시 서초구 서초1동,아이오닉5 (IONIQ5),전기
46396,202202,서울특별시 서초구 서초1동,Model 3 Long Range,전기


In [48]:
# null 확인. 여기는 null인 데이터가 없다.
el_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41673 entries, 6 to 189584
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기준년월    41673 non-null  int64 
 1   주소      41673 non-null  object
 2   차명      41673 non-null  object
 3   연료      41673 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.6+ MB


In [49]:
# 주소에서 구를 추출한다. 
el_car["구"] = el_car["주소"].map(lambda x : x.split()[1])
el_car.sample(3)

,기준년월,주소,차명,연료,구
42492,202202,서울특별시 중구 명동,아이오닉5 (IONIQ5),전기,중구
102459,202202,서울특별시 강남구 대치1동,아이오닉5 (IONIQ5),전기,강남구
113166,202202,서울특별시 강남구 역삼1동,Model S 75D,전기,강남구


#### 서울시 구별 전기차 수 

In [50]:
el_car = el_car.groupby("구")["차명"].count().sort_values(ascending=False).to_frame()
el_car = el_car.rename(columns={"차명" : "전기차수"})
el_car

,전기차수
구,
강남구,12173
구로구,3799
서초구,3495
영등포구,2388
송파구,1988
강서구,1776
성동구,1461
강동구,1293
마포구,1086


### 서울시 전기차 충전소 데이터

In [52]:
ev_raw = pd.read_excel("data/서울시충전소데이터.xlsx")
print(ev_raw.shape)
ev_raw.head(2)

(29561, 6)


,운영기관,충전소,지역,시군구,주소,충전용량
0,환경부(한국자동차환경협회),DMC 산학협력연구센터,서울특별시,마포구,서울특별시 마포구 매봉산로 37,급속(100kW멀티)
1,환경부(한국자동차환경협회),Park M (주차복합빌딩),서울특별시,마포구,서울특별시 마포구 매봉산로 80,급속(200kW동시)


#### 구별 전기차 충전소 갯수

In [53]:
ev_df = ev_raw.groupby("시군구")["충전소"].count().sort_values(ascending=False).to_frame()
# index 이름을 시군구에서 구로 변경
ev_df.index = ev_df.index.set_names("구")
# index 공백제거
ev_df.index = ev_df.index.str.strip()
ev_df

,충전소
구,
강남구,2252
송파구,2106
서초구,1947
성동구,1756
강서구,1596
성북구,1483
마포구,1410
노원구,1353
구로구,1344


### 서울시 구별 총 인구 데이터

In [5]:
people = pd.read_csv("data/서울시총인구데이터.csv", encoding="cp949")
people.head(3)

,행정구역별(읍면동),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,...,2021.10,2021.11,2021.12,2021.13,2021.14,2021.15,2021.16,2021.17,2021.18,2021.19
0,행정구역별(읍면동),총인구 (명),남자 (명),여자 (명),내국인-계 (명),내국인-남자 (명),내국인-여자 (명),외국인-계 (명),외국인-남자 (명),외국인-여자 (명),...,일반가구 (가구),집단가구 (가구),외국인가구 (가구),주택-계 (호),단독주택 (호),아파트 (호),연립주택 (호),다세대주택 (호),비거주용 건물내 주택 (호),주택이외의 거처 (호)
1,서울특별시,9472127,4584251,4887876,9125047,4421687,4703360,347080,162564,184516,...,4046799,2454,141918,3068494,299473,1818214,110239,812403,28165,270052
2,송파구,634720,305449,329271,624977,300957,324020,9743,4492,5251,...,259378,64,3828,215731,7791,131487,4720,70238,1495,18007


In [7]:
# 필요한 컬럼과 row 만 남긴다. 
people = people[["행정구역별(읍면동)", "2021"]].iloc[2:]
people

,행정구역별(읍면동),2021
2,송파구,634720
3,강서구,559837
4,노원구,503929
5,강남구,503019
6,관악구,495777
7,은평구,457385
8,강동구,451099
9,성북구,435509
10,양천구,433373
11,구로구,426220


#### 구별 인구 데이터

In [8]:
# 컬러명&인덱스 변경
people = people.rename(columns={"행정구역별(읍면동)":"구","2021":"인구"})
people = people.set_index("구")
people

,인구
구,
송파구,634720
강서구,559837
노원구,503929
강남구,503019
관악구,495777
은평구,457385
강동구,451099
성북구,435509
양천구,433373


In [9]:
# 인구 데이터가 object 타입이다.
people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 송파구 to 중구
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   인구      25 non-null     object
dtypes: object(1)
memory usage: 400.0+ bytes


In [11]:
# 인구를 숫자 타입으로 변경한다. 
people["인구"] = people["인구"].astype(int)
people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 송파구 to 중구
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   인구      25 non-null     int32
dtypes: int32(1)
memory usage: 300.0+ bytes


### 서울시 주유소 데이터

In [12]:
oil = pd.read_csv("data/서울시주유소데이터.csv", encoding="cp949")
oil.head()

,연번,자치구명,주유소명,주소
0,1,용산구,현대오일뱅크(주) 직영소월길주유소,서울특별시 용산구 소월로66
1,2,용산구,선익상사(주) 동자동주유소,서울특별시 용산구 한강대로 104길 6
2,3,용산구,현대오일뱅크㈜ 직영갈월동주유소,서울특별시 용산구 한강대로 322
3,4,용산구,서계주유소,서울특별시 용산구 청파로 367
4,5,용산구,㈜영원에너지 풍기주유소,서울특별시 용산구 원효로178


In [17]:
# 구별로 주유소 갯수를 구한다. 
oil_df = oil.groupby("자치구명")[["주유소명"]].count()
oil_df.head()

,주유소명
자치구명,
강남구,35
강동구,14
강북구,13
강서구,32
관악구,15


In [19]:
# 컬럼&인덱스명 변경
oil_df.index = oil_df.index.set_names("구")
oil_df = oil_df.rename(columns={"주유소명":"주유소갯수"})
oil_df.head()

,주유소갯수
구,
강남구,35
강동구,14
강북구,13
강서구,32
관악구,15


### 서울시 휘발유 자동차 데이터

In [28]:
car_raw = pd.read_csv("data/서울시자동차등록현황.csv", encoding="cp949")
# 처음 빈칸인 4줄은 생략
car_raw = car_raw[4:]
car_raw.sample(2)

,행정동-연료별 분류,CNG,경유,기타연료,수소,엘피지,전기,하이브리드(CNG-전기),하이브리드(LPG-전기),하이브리드(경유-전기),하이브리드(휘발유-전기),휘발유,휘발유(무연),휘발유(유연),총합계
400,서울특별시 종로구 부암동,NaN,1041.00000,5.00000,2.00000,148.00000,30.00000,NaN,6.00000,3.00000,163.00000,785.00000,995.00000,1.00000,3179
213,서울특별시 마포구 대흥동,NaN,1252.00000,5.00000,3.00000,267.00000,21.00000,NaN,1.00000,NaN,184.00000,945.00000,1223.00000,2.00000,3903


In [38]:
# 주소 데이터에서 구 추출하기
car_raw["구"] = car_raw["행정동-연료별 분류"].map(lambda x : x.split()[1].strip())
# 휘발유 자동차만 추출한다. 
car_df = car_raw[["구","경유","하이브리드(경유-전기)","하이브리드(휘발유-전기)","휘발유","휘발유(무연)","휘발유(유연)"]]
car_df

,구,경유,하이브리드(경유-전기),하이브리드(휘발유-전기),휘발유,휘발유(무연),휘발유(유연)
4,강남구,1178.00000,1.00000,190.00000,917.00000,1058.00000,2.00000
5,강남구,2714.00000,10.00000,624.00000,2949.00000,2947.00000,5.00000
6,강남구,2722.00000,6.00000,464.00000,2230.00000,2612.00000,3.00000
7,강남구,3608.00000,9.00000,416.00000,2661.00000,3309.00000,3.00000
8,강남구,3646.00000,15.00000,376.00000,2964.00000,2791.00000,4.00000
...,...,...,...,...,...,...,...
448,중랑구,1944.00000,4.00000,166.00000,1008.00000,1787.00000,3.00000
449,중랑구,4806.00000,6.00000,421.00000,2126.00000,3821.00000,3.00000
450,중랑구,2370.00000,2.00000,219.00000,1157.00000,2130.00000,3.00000
451,중랑구,2112.00000,2.00000,142.00000,965.00000,1800.00000,2.00000


In [39]:
# 결측치 확인
car_df.isnull().sum()

구                 0
경유                3
하이브리드(경유-전기)     75
하이브리드(휘발유-전기)    22
휘발유              12
휘발유(무연)           9
휘발유(유연)          89
dtype: int64

In [40]:
# 결측치를 0으로 변경
car_df = car_df.fillna(0)

In [41]:
# 구별로 각 연료 자동차 별 합 구하기
car_df = car_df.groupby("구").sum()
car_df

,경유,하이브리드(경유-전기),하이브리드(휘발유-전기),휘발유,휘발유(무연),휘발유(유연)
구,,,,,,
강남구,68250.00000,286.00000,15256.00000,65853.00000,66798.00000,89.00000
강동구,51743.00000,87.00000,6512.00000,30220.00000,47616.00000,57.00000
강북구,27457.00000,23.00000,2119.00000,13196.00000,23432.00000,20.00000
강서구,69132.00000,96.00000,10387.00000,38463.00000,63633.00000,66.00000
관악구,41324.00000,36.00000,4327.00000,22432.00000,38518.00000,55.00000
광진구,36063.00000,90.00000,3690.00000,20229.00000,29805.00000,32.00000
구로구,53887.00000,43.00000,4847.00000,24688.00000,44649.00000,42.00000
금천구,36609.00000,49.00000,2784.00000,14130.00000,27535.00000,39.00000
노원구,49714.00000,39.00000,5446.00000,28313.00000,50268.00000,48.00000


In [42]:
# 구별 합이 필요하여, row 기준으로 합한 데이터를 구한다. 
car_df["휘발유자동차수"] = car_df.sum(axis=1).astype(int)
car_df.head(2)

,경유,하이브리드(경유-전기),하이브리드(휘발유-전기),휘발유,휘발유(무연),휘발유(유연),휘발유자동차수
구,,,,,,,
강남구,68250.00000,286.00000,15256.00000,65853.00000,66798.00000,89.00000,216532
강동구,51743.00000,87.00000,6512.00000,30220.00000,47616.00000,57.00000,136235


#### 구별 휘발유 자동차 수

In [43]:
car_df = car_df[["휘발유자동차수"]]
car_df

,휘발유자동차수
구,
강남구,216532
강동구,136235
강북구,66247
강서구,181777
관악구,106692
광진구,89909
구로구,128156
금천구,81146
노원구,133828


### 전체 데이터 합치기

In [56]:
# 전기차 & 충전소
df = el_car.merge(ev_df, left_index=True, right_index=True)
df

,전기차수,충전소
구,,
강남구,12173,2252
구로구,3799,1344
서초구,3495,1947
영등포구,2388,1228
송파구,1988,2106
강서구,1776,1596
성동구,1461,1756
강동구,1293,1272
마포구,1086,1410


In [57]:
# 인구
df = df.merge(people, left_index=True, right_index=True)
# 주유소 갯수
df = df.merge(oil_df, left_index=True, right_index=True)
# 휘발유 자동차 수
df = df.merge(car_df, left_index=True, right_index=True)
df

,전기차수,충전소,인구,주유소갯수,휘발유자동차수
구,,,,,
강남구,12173,2252,503019,35,216532
구로구,3799,1344,426220,21,128156
서초구,3495,1947,392302,33,166410
영등포구,2388,1228,402984,29,133605
송파구,1988,2106,634720,33,226642
강서구,1776,1596,559837,32,181777
성동구,1461,1756,286469,17,96307
강동구,1293,1272,451099,14,136235
마포구,1086,1410,365192,12,111313
